## Locations extraction, Series I

### Part 1: Regex matching

In [191]:
import pandas as pd
import requests
import json

In [193]:
doc_catalog = pd.read_csv('document-catalog-07112025.csv')
uscities = pd.read_csv('uscities.csv')

In [305]:
doc_catalog[(doc_catalog['ID'] >= 1) & (doc_catalog['ID'] <= 700) & (doc_catalog['Workflow Stage'] == 'Cataloged')].sort_values(by = 'ID').head(30)

,ID,Box,title,Document Date,Document Length,Manuscript Type,Collection,Source,Internal Notes,Workflow Stage,...,Document Body,Signature,Post Script,Document Extra,Marginalia,Enclosure,Footnote,General,Image Identifier,Response
1752,110,Box 1 Folder 25,Duplicate of 458: Address at the Dedication of...,"Sat, 10/18/1969 - 12:00",6.0,NaN,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate record that we'll delete after confi...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1771,130,Box 4 Folder 17,Duplicate of ID120: Speech concerning South Af...,"Mon, 01/01/1973 - 12:00\n - Mon, 12/31/1973 - ...",16.0,"Typed, Copy",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID120. Do not publish this record...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1770,133,Box 4 Folder 4,Duplicate of ID81: Speech to the Racial Harmo...,"Sat, 01/01/1972 - 12:00\n - Sun, 12/31/1972 - ...",15.0,"Typed, Draft",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID81. Do not publish this record ...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1769,134,Box 4 Folder 4,"Duplicate of ID82: Draft, Speech to Racial Har...","Sat, 01/01/1972 - 12:00\n - Sun, 12/31/1972 - ...",15.0,"Typed, Draft",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID82. Do not publish this record ...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1768,135,Box 4 Folder 4,"Duplicate of ID310: Du Bois ""Credo,"" 1904",NaN,2.0,"Typed, Copy",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID310. Do not publish this record...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1799,138,Box 4 Folder 7,Duplicate of ID86: Speech at Press Conference ...,"Tue, 01/02/1973 - 12:00",6.0,"Handwritten, Typed, Draft",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID86. Do not publish this record ...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1798,139,Box 4 Folder 7,Duplicate of ID87: May the White Blossom of Ba...,"Mon, 01/01/1973 - 12:00\n - Mon, 12/31/1973 - ...",2.0,"Typed, Document",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID87. Do not publish this record ...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1797,140,Box 4 Folder 7,"Duplicate of ID87: Diplomats, newsmen see ruin...","Sat, 12/30/1972 - 12:00",1.0,Printed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID87. Do not publish this record ...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1796,141,Box 4 Folder 7,Duplicate of ID87: Press Conference Participan...,"Wed, 01/03/1973 - 12:00",1.0,"Typed, Document",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID87. Do not publish this record ...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1795,142,Box 4 Folder 7,Duplicate of ID86: Largest Hospital in Hanoi R...,"Sat, 12/23/1972 - 12:00",1.0,Typed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Duplicate of ID86. Do not publish this record ...,Cataloged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [287]:
drop_cols = ['Document Date', 'Manuscript Type', 'Collection', 'Source', 'Internal Notes',
       'Object Type', 'Author', 'Keywords', 'Head',
       'Document Top', 'Dateline', 'Salutation', 'Document Body', 'Signature',
       'Post Script', 'Document Extra', 'Marginalia', 'Enclosure', 'Footnote',
       'General', 'Image Identifier', 'Response']
missing_docs = missing_docs.drop(drop_cols, axis = 1)

In [289]:
missing_docs.to_csv('missing_docs.csv')

In [195]:
early_access = doc_catalog[doc_catalog['Workflow Stage'] == 'Early Access']

In [197]:
early_access.columns

Index(['ID', 'Box', 'title', 'Document Date', 'Document Length',
       'Manuscript Type', 'Collection', 'Source', 'Internal Notes',
       'Workflow Stage', 'Object Type', 'Author', 'Keywords', 'Head',
       'Document Top', 'Dateline', 'Salutation', 'Document Body', 'Signature',
       'Post Script', 'Document Extra', 'Marginalia', 'Enclosure', 'Footnote',
       'General', 'Image Identifier', 'Response'],
      dtype='object')

In [199]:
early_access['Document Date'].iloc[0]

'Thu, 06/28/1984 - 12:00'

In [201]:
import datetime
import dateutil.parser as parser

In [203]:
dt = parser.parse(early_access['Document Date'].iloc[0])
dt.year

1984

In [205]:
early_access['date_mod'] = early_access.apply(lambda row: row['Document Date'].split('-')[0].strip(' '), axis=1)
early_access['year'] = early_access.apply(lambda row: parser.parse(row['date_mod']).year, axis=1)

C:\Users\charl\AppData\Local\Temp\ipykernel_35760\810399839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  early_access['date_mod'] = early_access.apply(lambda row: row['Document Date'].split('-')[0].strip(' '), axis=1)
C:\Users\charl\AppData\Local\Temp\ipykernel_35760\810399839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  early_access['year'] = early_access.apply(lambda row: parser.parse(row['date_mod']).year, axis=1)


In [207]:
early_access['year'].value_counts()

year
1975    58
1977    48
1971    39
1976    34
1969    32
1972    30
1973    20
1968    16
1970    16
1998    12
1979    11
1988    11
1985    11
1967    10
2004     9
1978     9
1995     9
1990     8
2003     8
2002     8
1992     8
1980     7
2000     7
1991     7
2001     7
1981     6
1993     6
1984     5
1989     5
1999     5
1996     5
1997     5
1974     4
1982     4
1986     3
2005     3
1994     2
1966     2
1987     2
1983     2
1965     2
1964     1
1958     1
1945     1
Name: count, dtype: int64

In [183]:
import numpy as np
np.__version__

'2.3.0'

In [185]:
import spacy

nlp = spacy.load("en_core_web_sm")
text = "I visited Paris, France, and then hiked in the Alps."
doc = nlp(text)

locations = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]]
print(locations)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [219]:
early_access.columns

Index(['ID', 'Box', 'title', 'Document Date', 'Document Length',
       'Manuscript Type', 'Collection', 'Source', 'Internal Notes',
       'Workflow Stage', 'Object Type', 'Author', 'Keywords', 'Head',
       'Document Top', 'Dateline', 'Salutation', 'Document Body', 'Signature',
       'Post Script', 'Document Extra', 'Marginalia', 'Enclosure', 'Footnote',
       'General', 'Image Identifier', 'Response', 'date_mod', 'year'],
      dtype='object')

In [239]:
letters.Salutation

1755                                         Dear Julian:
1811                                   Dear Senator Bond:
1848                                                  NaN
1853                                                  NaN
1905                                                  NaN
2014                                                  NaN
2260                                                  NaN
2374    Miss Sonya Moulder\nEdmund Asa Ware Elementary...
2408    Mr. Julian Bond:\nHouse of Representative\nGeo...
2677                                     Dear Mr. Hughes:
2882                                       Dear Dr. King:
2890                                           Gentlemen:
3389    Mr. Julian Bond\n162 Ehuralee Street, S. W.\nA...
3391    Dr. Hugh Gloster, President\nMorehouse College...
5558                                    Dear Miss Bolton,
5559                                     Dear Ms. Chally,
5560                                    Dear Miss Mooney,
5561          

In [249]:
letters = early_access[early_access['Object Type'] == 'Letter']

storage = {}
for idx, row in letters.iterrows():
    places = []
    for field in ['Dateline', 'Salutation']:
        text = row[field]
        if pd.notnull(text):
            hits = re.findall(pattern, text, flags=re.IGNORECASE)
            places.append(hits) if hits else None
    storage[idx] = places
        

In [251]:
storage

{1755: [],
 1811: [],
 1848: [],
 1853: [],
 1905: [],
 2014: [],
 2260: [],
 2374: [['Atlanta, Georgia']],
 2408: [['Atlanta, Georgia'], ['Atlanta, Georgia']],
 2677: [['New York, New York']],
 2882: [['Atlanta, Georgia']],
 2890: [['Niagara Falls, New York']],
 3389: [['Atlanta, Georgia'], ['Atlanta, Georgia']],
 3391: [['Atlanta, Georgia']],
 5558: [['Atlanta, Georgia']],
 5559: [['Morris, Illinois']],
 5560: [['Decatur, Georgia']],
 5561: [],
 5563: [['Atlanta, Georgia']]}

In [253]:
# can work with that. Still don't love using regex though. But let's call it a day.

In [119]:
uscities.head(50)

,city,state_name,lat,lng,zips,id
0,New York,New York,40.6943,-73.9249,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,California,34.1141,-118.4068,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Illinois,41.8375,-87.6866,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Florida,25.7840,-80.2101,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Houston,Texas,29.7860,-95.3885,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925
5,Dallas,Texas,32.7935,-96.7667,75098 75287 75230 75231 75236 75237 75235 7525...,1840019440
6,Philadelphia,Pennsylvania,40.0077,-75.1339,19154 19151 19150 19153 19152 19102 19103 1910...,1840000673
7,Atlanta,Georgia,33.7628,-84.4220,30334 30331 30332 30309 30308 30354 30327 3032...,1840013660
8,Washington,District of Columbia,38.9047,-77.0163,20250 20010 20011 20012 20015 20520 20418 2041...,1840006060
9,Boston,Massachusetts,42.3188,-71.0852,02120 02121 02122 02124 02125 02126 02127 0212...,1840000455


In [189]:
# zips poses interesting possibilities for doing location tagging for letters, but I fear that it may be time-consuming.

In [225]:
state_abbrev_to_name = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas",
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware",
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas",
    "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada",
    "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York",
    "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah",
    "VT": "Vermont", "VA": "Virginia", "WA": "Washington", "WV": "West Virginia",
    "WI": "Wisconsin", "WY": "Wyoming", "D.C.": "District of Columbia"
}

state_name_to_abbrev = {v: k for k, v in state_abbrev_to_name.items()}

uscities['state_abbrev'] = uscities['state_name'].map(state_name_to_abbrev)

# Step 3: Generate all combinations
location_variants = set()

for _, row in uscities.iterrows():
    city = row['city']
    state = row['state_name']
    abbrev = row['state_abbrev']
    
    # location_variants.add(city)
    location_variants.add(f"{city}, {state}")
    location_variants.add(f"{city}, {abbrev}")

# Step 4: Escape and build regex pattern
escaped_locations = [re.escape(loc) for loc in location_variants]
pattern = r'\b(?:' + '|'.join(escaped_locations) + r')\b'

# Step 5: Test text
title = "Top Tech Events in San Francisco, Austin, and Miami, FL"

# Find matches
matches = re.findall(pattern, title, flags=re.IGNORECASE)
print(matches)

['Miami, FL']


In [137]:
early_access['potential_matches'] = early_access.apply(lambda row: re.findall(pattern, row['title'], flags=re.IGNORECASE), axis=1)

C:\Users\charl\AppData\Local\Temp\ipykernel_35760\3648656043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  early_access['potential_matches'] = early_access.apply(lambda row: re.findall(pattern, row['title'], flags=re.IGNORECASE), axis=1)


In [167]:
matches = pd.DataFrame([(row['potential_matches'],row['title'],row['ID'],row['year']) for idx, row in early_access.iterrows() if row['potential_matches']])

In [143]:
# makes me sadddddd how do I fix also why is Washington, D.C. still not showing up ffs

In [177]:
matches[matches[3] == 1970]

,0,1,2,3
93,"[Atlantic City, New Jersey]",Speech given at the Philadelphia Bar Associati...,470,1970


In [179]:
early_access[early_access['year'] == 1970]

,ID,Box,title,Document Date,Document Length,Manuscript Type,Collection,Source,Internal Notes,Workflow Stage,...,Document Extra,Marginalia,Enclosure,Footnote,General,Image Identifier,Response,date_mod,year,potential_matches
1947,309,Box 4 Folder 14,Speech about Black Economic Development Dec 19...,"Tue, 12/01/1970 - 12:00\n - Thu, 12/31/1970 - ...",14.0,"Handwritten, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",There are three items in this folder. This ent...,Early Access,...,NaN,NaN,NaN,"1 Theodore L. Cross, Black Capitalism, p. 5 (1...",Speech concerning black people and economic st...,"2858886, 2858887, 2858888, 2858889, 2858890, 2...",NaN,"Tue, 12/01/1970",1970,[]
2111,470,Box 2 Folder 14,Speech given at the Philadelphia Bar Associati...,"Tue, 09/01/1970 - 12:00\n - Wed, 09/30/1970 - ...",10.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Early Access,...,NaN,NaN,NaN,"*Tom Hayden, Colonialism & Liberation, Politic...",NaN,"3202754, 3202755, 3202756, 3202757, 3202758, 3...",NaN,"Tue, 09/01/1970",1970,"[Atlantic City, New Jersey]"
2117,462,Box 2 Folder 8,"""Rush Towards Freedom"" Television Series, Corr...","Sat, 02/21/1970 - 12:00\n - Sun, 02/22/1970 - ...",50.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Note: Please scan all pages in this f...,Early Access,...,NaN,NaN,NaN,NaN,NaN,"3202782, 3202783, 3202784, 3202785, 3202786, 3...",NaN,"Sat, 02/21/1970",1970,[]
2118,461,Box 2 Folder 9,"Speech- ""The Nature of the Crisis in Health Ca...","Fri, 02/13/1970 - 12:00",10.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Early Access,...,NaN,NaN,NaN,NaN,NaN,"3202867, 3202868, 3202869, 3202870, 3202871, 3...",NaN,"Fri, 02/13/1970",1970,[]
2121,456,Box 2 Folder 10,"Background Paper for ""Speech to Doctors"" on Me...","Tue, 03/31/1970 - 12:00",10.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Early Access,...,NaN,NaN,NaN,NaN,"Speech- ""The Nature of the Crisis in Health Ca...","3202857, 3202858, 3202859, 3202860, 3202861, 3...",NaN,"Tue, 03/31/1970",1970,[]
2180,548,Box 11 Folder 12,"Speech concerning the crisis in black schools,...","Thu, 01/01/1970 - 12:00\n - Thu, 12/31/1970 - ...",21.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Early Access,...,NaN,NaN,NaN,"* Tom Hayden, Colonialism & Liberation, Politi...",NaN,"3243287, 3243288, 3243289, 3243290, 3243291, 3...",NaN,"Thu, 01/01/1970",1970,[]
2187,577,Box 1 Folder 15,Draft Speech 1970? (similar to Speech at the H...,"Thu, 01/01/1970 - 12:00\n - Tue, 12/01/1970 - ...",12.0,"Draft, Typed, Handwritten, Version",Papers of Julian Bond 1897-2006\n,"Special Collections, University of Virginia Li...",Scanning Notes: There are two items in this fo...,Early Access,...,NaN,NaN,NaN,NaN,"Speech at the Hungry Club Atlanta, February 12...","3201949, 3201950, 3201951, 3201952, 3201953, 3...",NaN,"Thu, 01/01/1970",1970,[]
2189,575,Box 11 Folder 29,"Speech- ""Today's Youth Tomorrow's Leaders "", n...","Thu, 01/01/1970 - 12:00\n - Thu, 12/31/1970 - ...",12.0,"Copy, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Early Access,...,NaN,NaN,NaN,"1 Richard J. Barnet, The Economy of Death\n, 2...","Speech about race and war in American life, 19...","3242964, 3242965, 3242966, 3242967, 3242968, 3...",NaN,"Thu, 01/01/1970",1970,[]
2191,572,Box 11 Folder 27,COPY: Speech concerning the crisis in black sc...,"Thu, 01/01/1970 - 12:00\n - Thu, 12/31/1970 - ..

In [7]:
doc_catalog.drop(['Box', 'Document Length', 'Workflow Stage',
                  'Image Filename', 'Image Identifier', 'Image URL'], axis=1, inplace=True)

In [9]:
doc_catalog

,ID,Title,Document Body
0,670,Undated Speech concerning Conditions of Black ...,[This speech includes pages with many differen...
1,667,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o..."
2,666,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o..."
3,665,Speeches making observations about the recent ...,"1\nNow that the nations voters — at least, 54%..."
4,663,Speech about the upcoming presidential electio...,The election approaching on November seventh i...
...,...,...,...
235,83,Speech concerning black bankers and businesses...,"Before I begin, let me tell you my financial q..."
236,84,Speech concerning challenges facing the Americ...,Now that the nation has changed had a change o...
237,85,Speech from the Alabama Voter Education Projec...,Thank you a great deal. As John Lewis has told...
238,86,Speech at Press Conference in Washington conce...,[Handwritten.]\none the point of today's conce...


In [19]:
doc_catalog_ext = pd.read_xml('1970s/document-catalog.csv.xml')

In [25]:
def clean_dataframe(df):
    df = df.replace(r'\\r|\\n|\\t|\n', '', regex=True)
    df = df.replace(r'<[^>]*>', '', regex=True)
    return df

clean_dataframe(doc_catalog_ext)

,key,nid,field_box,title,field_date,field_document_length,field_manuscript_type,field_collection,field_source,field_internal_notes,...,field_document_body,field_signature,field_post_script,field_document_extra,field_marginalia,field_enclosure,field_footnote,field_general,field_image_identifier,field_response
0,0,8277,Box 17 Folder 2,From Julian Bond Memo Concerning the Westbrook...,None,1,"Copy, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...","Bulk imported. no date clues, but it is in the...",...,None,None,None,None,None,None,None,From Julian Bond Memo Concerning the Westbrook...,None,None
1,1,8276,Box 17 Folder 2,From Julian Bond Memo Concerning the Westbrook...,None,1,"Copy, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...","Bulk imported. no date clues, but it is in the...",...,None,None,None,None,None,None,None,None,None,None
2,2,8275,Box 17 Folder 2,To Julian Bond from Dr. Richard Allen Williams...,"Thu, 11/29/1973 - 12:00",1,"Letterhead, Signed, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Bulk imported. no response attached,...,None,None,None,None,None,None,None,None,None,None
3,3,8274,Box 17 Folder 2,[Fragment] From Julian Bond to Cary Internatio...,None,1,"Copy, Draft, Typed, Fragment",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...","Bulk imported. no date clues, but it is in the...",...,None,None,None,None,None,None,None,None,None,None
4,4,8273,Box 17 Folder 2,"To Julian Bond from Panke Bradley, Mrs. L. Fit...","Mon, 09/30/1974 - 12:00",2,"Letterhead, Handwritten, Signed, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Bulk imported. Scanning note: please remove th...,...,None,None,None,None,None,None,None,None,None,"From Julian Bond to Panke Bradley, 23 Oct [1974]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8225,225,483,None,Home,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8226,226,482,None,Press Coverage,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8227,227,481,None,Editorial Method,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8228,228,480,None,Julian Bond,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [27]:
full_catalog = clean_dataframe(doc_catalog_ext)
full_catalog.columns

Index(['key', 'nid', 'field_box', 'title', 'field_date',
       'field_document_length', 'field_manuscript_type', 'field_collection',
       'field_source', 'field_internal_notes', 'moderation_state',
       'field_object_type', 'field_author', 'field_keywords', 'field_head',
       'field_document_top', 'field_dateline', 'field_salutation',
       'field_document_body', 'field_signature', 'field_post_script',
       'field_document_extra', 'field_marginalia', 'field_enclosure',
       'field_footnote', 'field_general', 'field_image_identifier',
       'field_response'],
      dtype='object')

In [31]:
full_catalog.moderation_state.unique()

array(['Cataloged', 'Early Access', 'Digitized', 'Transcribed',
       'Proof read', 'Tandem read', 'Draft', None], dtype=object)

In [37]:
full_catalog.field_object_type.unique()

array(['Letter', 'Article', None, 'Speech', 'Essay', 'Ephemera',
       'Interview', 'Report', 'Pamphlet', 'Legal'], dtype=object)

In [55]:
speeches = full_catalog[full_catalog['field_object_type']=='Speech']
speeches = speeches[speeches['moderation_state']=='Early Access']

In [61]:
uscities.drop(['zips', 'id'], axis=1, inplace=True)

In [63]:
uscities

,city,state_name,lat,lng
0,New York,New York,40.6943,-73.9249
1,Los Angeles,California,34.1141,-118.4068
2,Chicago,Illinois,41.8375,-87.6866
3,Miami,Florida,25.7840,-80.2101
4,Houston,Texas,29.7860,-95.3885
...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032
31116,Ironville,Pennsylvania,40.6586,-78.2155
31117,Newkirk,New Mexico,35.0635,-104.2715
31118,Falcon Village,Texas,26.5652,-99.1341


In [65]:
uscities['City'] = uscities['city'] + ', ' + uscities['state_name']

In [71]:
cities = list(uscities.City)
city_pattern = '|'.join([f'\\b{city}\\b' for city in cities])

# Extract the city name using str.extract()
speeches['city'] = speeches['title'].str.extract(f'({city_pattern})', expand=False)

speeches

,key,nid,field_box,title,field_date,field_document_length,field_manuscript_type,field_collection,field_source,field_internal_notes,...,field_signature,field_post_script,field_document_extra,field_marginalia,field_enclosure,field_footnote,field_general,field_image_identifier,field_response,city
1340,340,347,Box 7 Folder 30,Speech attacking the Reagan Administration&#03...,"Thu, 06/28/1984 - 12:00",12,"Handwritten, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",There are two items in this folder. This entry...,...,None,None,None,None,None,None,None,"3205197, 3205198, 3205199, 3205200, 3205201, 3...",None,NaN
1341,341,346,Box 7 Folder 30,Speech attacking the Reagan Administration&#03...,"Thu, 06/28/1984 - 12:00",13,"Handwritten, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",NEEDS REVIEW: Needs UVA IIIF -- sent images bu...,...,None,None,None,None,None,None,Speech attacking the Reagan Administration&#03...,None,None,NaN
1342,342,379,Box 8 Folder 18,Paper- &quot;The Movement and the Media: An An...,"Tue, 04/17/1990 - 12:00",19,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",There is one item in this folder.Genre note: T...,...,None,None,None,None,None,"1. Quoted in The New York Times, March 18, 199...",None,"3205955, 3205956, 3205957, 3205958, 3205959, 3...",None,NaN
1343,343,88,Box 4 Folder 8,Speech on the health of the United States educ...,"Fri, 02/02/1973 - 12:00",29,None,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",6/7/22 - Reviewed by KZantowScanning Notes:Jus...,...,None,None,None,None,None,"1 Who Cares&nbsp;About Education?"" Craig S. Ba...",None,"2857526, 2857527, 2857528, 2857529, 2857530, 2...",None,NaN
1352,352,86,Box 4 Folder 7,Speech at Press Conference in Washington conce...,"Tue, 01/02/1973 - 12:00",9,"Type, Handwritten, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes:Box 4 Folder 7 contains two doc...,...,None,None,"THE NEW YORK TIMES, SATURDAY, DECEMBER 23, 197...",[The title and description of this document ar...,None,None,None,"2857508, 2857509, 2857510, 2857511, 2857512, 2...",None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900,900,434,Box 9 Folder 30,"Speech- &quot;The 1965 Voting Rights Act, A Be...","Fri, 10/02/1998 - 12:00",18,"Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",NEEDS REVIEW: Needs UVA IIIF -- sent images bu...,...,None,None,None,None,None,"1 President Lyndon B. Johnson, March 15, 1965....","Speech- &quot;The 1965 Voting Rights Act, A Be...",None,None,NaN
1902,902,503,Box 10 Folder 9,"Commencement Address at Washington University,...","Fri, 05/19/2000 - 12:00",9,Typed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",NEEDS REVIEW: Needs UVA IIIF -- sent images bu...,...,None,None,None,None,None,"1. Bond, James, Commencement Address, Berea Co...","Commencement Address at Washington University,...",None,None,NaN
1903,903,518,Box 10 Folder 20,"Speech- &quot;Public Intellectuals &quot;, 12 ...","Fri, 04/12/2002 - 12:00",7,"Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",NEEDS REVIEW: IIIF did not get created; report...,...,None,None,None,None,None,"1 Mills, C. Wright, White Collar: the American...",None,None,None,NaN
1904,904,566,Box 11 Folder 23,Speech concerning politics and the debate surr...,"Wed, 01/01/1969 - 12:00",13,"Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",NEEDS REVIEW: Needs UVA IIIF -- sent images bu...,...,None,None,None,None,None,"* Dr. Kenneth Clark, 1967 speech to the Southe...",(Draft) Speech concerning politics and the deb...,None,None,NaN


In [135]:
speeches.to_excel('speeches.xlsx')

In [137]:
speeches = pd.read_excel('speeches.xlsx')

In [139]:
speeches

,Unnamed: 0,key,nid,field_box,title,field_date,field_document_length,field_manuscript_type,field_collection,field_source,...,field_signature,field_post_script,field_document_extra,field_marginalia,field_enclosure,field_footnote,field_general,field_image_identifier,field_response,city
0,1340,340,347,Box 7 Folder 30,Speech attacking the Reagan Administration&#03...,"Thu, 06/28/1984 - 12:00",12,"Handwritten, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3205197, 3205198, 3205199, 3205200, 3205201, 3...",NaN,"Boston, MA"
1,1341,341,346,Box 7 Folder 30,Speech attacking the Reagan Administration&#03...,"Thu, 06/28/1984 - 12:00",13,"Handwritten, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,NaN,Speech attacking the Reagan Administration&#03...,NaN,NaN,"Boston, MA"
2,1342,342,379,Box 8 Folder 18,Paper- &quot;The Movement and the Media: An An...,"Tue, 04/17/1990 - 12:00",19,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,"1. Quoted in The New York Times, March 18, 199...",NaN,"3205955, 3205956, 3205957, 3205958, 3205959, 3...",NaN,"Cambridge, MA"
3,1343,343,88,Box 4 Folder 8,Speech on the health of the United States educ...,"Fri, 02/02/1973 - 12:00",29,NaN,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,"1 Who Cares&nbsp;About Education?"" Craig S. Ba...",NaN,"2857526, 2857527, 2857528, 2857529, 2857530, 2...",NaN,NaN
4,1352,352,86,Box 4 Folder 7,Speech at Press Conference in Washington conce...,"Tue, 01/02/1973 - 12:00",9,"Type, Handwritten, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,"THE NEW YORK TIMES, SATURDAY, DECEMBER 23, 197...",[The title and description of this document ar...,NaN,NaN,NaN,"2857508, 2857509, 2857510, 2857511, 2857512, 2...",NaN,"Washington, DC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,1900,900,434,Box 9 Folder 30,"Speech- &quot;The 1965 Voting Rights Act, A Be...","Fri, 10/02/1998 - 12:00",18,"Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,"1 President Lyndon B. Johnson, March 15, 1965....","Speech- &quot;The 1965 Voting Rights Act, A Be...",NaN,NaN,NaN
383,1902,902,503,Box 10 Folder 9,"Commencement Address at Washington University,...","Fri, 05/19/2000 - 12:00",9,Typed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,"1. Bond, James, Commencement Address, Berea Co...","Commencement Address at Washington University,...",NaN,NaN,"St. Louis, MO"
384,1903,903,518,Box 10 Folder 20,"Speech- &quot;Public Intellectuals &quot;, 12 ...","Fri, 04/12/2002 - 12:00",7,"Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,"1 Mills, C. Wright, White Collar: the American...",NaN,NaN,NaN,NaN
385,1904,904,566,Box 11 Folder 23,Speech concerning politics and the debate surr...,"Wed, 01/01/1969 - 12:00",13,"Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",...,NaN,NaN,NaN,NaN,NaN,"* Dr. Kenneth Clark, 1967 speech to the Southe...",(Draft) Speech concerning politics and the deb...,NaN,NaN,NaN


In [141]:
sum(speeches.city.value_counts())

161

In [113]:
# doc_catalog['city_from_body'] = doc_catalog['Document Body'].str.extract(f'({city_pattern})', expand=False)

# print(doc_catalog)

In [114]:
# sum(doc_catalog.city_from_body.value_counts())

In [143]:
speeches.columns

Index(['Unnamed: 0', 'key', 'nid', 'field_box', 'title', 'field_date',
       'field_document_length', 'field_manuscript_type', 'field_collection',
       'field_source', 'field_internal_notes', 'moderation_state',
       'field_object_type', 'field_author', 'field_keywords', 'field_head',
       'field_document_top', 'field_dateline', 'field_salutation',
       'field_document_body', 'field_signature', 'field_post_script',
       'field_document_extra', 'field_marginalia', 'field_enclosure',
       'field_footnote', 'field_general', 'field_image_identifier',
       'field_response', 'city'],
      dtype='object')

In [147]:
drop_cols = ['Unnamed: 0', 'field_box', 'field_date',
       'field_document_length', 'field_manuscript_type', 'field_collection',
       'field_source', 'field_internal_notes', 'moderation_state',
       'field_object_type', 'field_head',
       'field_document_top', 'field_dateline', 'field_salutation',
       'field_document_body', 'field_signature', 'field_post_script',
       'field_document_extra', 'field_marginalia',
       'field_footnote', 'field_image_identifier']
speeches.drop(drop_cols, axis=1, inplace=True)

In [159]:
speeches.drop('field_response', axis=1, inplace=True)

In [165]:
import re

In [169]:
state_abbrev_to_name = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas",
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware",
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas",
    "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada",
    "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York",
    "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah",
    "VT": "Vermont", "VA": "Virginia", "WA": "Washington", "WV": "West Virginia",
    "WI": "Wisconsin", "WY": "Wyoming", "DC": "District of Columbia"
}

def replace_state_abbreviations(text):
    # Compile a regex pattern that matches all two-letter state abbreviations as whole words
    pattern = r'\b(' + '|'.join(state_abbrev_to_name.keys()) + r')\b'
    return re.sub(pattern, lambda m: state_abbrev_to_name[m.group()], text)

speeches.city = speeches.city.fillna('').apply(replace_state_abbreviations)
speeches

,key,nid,title,field_author,field_keywords,field_enclosure,field_general,city
0,340,347,Speech attacking the Reagan Administration&#03...,"Bond, Julian",Race and Law,NaN,NaN,"Boston, Massachusetts"
1,341,346,Speech attacking the Reagan Administration&#03...,"Bond, Julian",Race and Law,NaN,Speech attacking the Reagan Administration&#03...,"Boston, Massachusetts"
2,342,379,Paper- &quot;The Movement and the Media: An An...,"Bond, Julian","Education, Organizing",NaN,NaN,"Cambridge, Massachusetts"
3,343,88,Speech on the health of the United States educ...,"Bond, Julian",NaN,NaN,NaN,
4,352,86,Speech at Press Conference in Washington conce...,"Bond, Julian",NaN,NaN,NaN,"Washington, District of Columbia"
...,...,...,...,...,...,...,...,...
382,900,434,"Speech- &quot;The 1965 Voting Rights Act, A Be...","Bond, Julian","Disenfranchisement, Race and Law",NaN,"Speech- &quot;The 1965 Voting Rights Act, A Be...",
383,902,503,"Commencement Address at Washington University,...","Bond, Julian",Education,NaN,"Commencement Address at Washington University,...","St. Louis, Missouri"
384,903,518,"Speech- &quot;Public Intellectuals &quot;, 12 ...","Bond, Julian","Education, Organizing",NaN,NaN,
385,904,566,Speech concerning politics and the debate surr...,"Bond, Julian","Economics, Organizing",NaN,(Draft) Speech concerning politics and the deb...,


In [171]:
merged = pd.merge(speeches, uscities, left_on = 'city', right_on='City', how='left')
# merged_pt_2.dropna(inplace=True)
merged

,key,nid,title,field_author,field_keywords,field_enclosure,field_general,city_x,city_y,state_name,lat,lng,City
0,340,347,Speech attacking the Reagan Administration&#03...,"Bond, Julian",Race and Law,NaN,NaN,"Boston, Massachusetts",Boston,Massachusetts,42.3188,-71.0852,"Boston, Massachusetts"
1,341,346,Speech attacking the Reagan Administration&#03...,"Bond, Julian",Race and Law,NaN,Speech attacking the Reagan Administration&#03...,"Boston, Massachusetts",Boston,Massachusetts,42.3188,-71.0852,"Boston, Massachusetts"
2,342,379,Paper- &quot;The Movement and the Media: An An...,"Bond, Julian","Education, Organizing",NaN,NaN,"Cambridge, Massachusetts",Cambridge,Massachusetts,42.3759,-71.1185,"Cambridge, Massachusetts"
3,343,88,Speech on the health of the United States educ...,"Bond, Julian",NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
4,352,86,Speech at Press Conference in Washington conce...,"Bond, Julian",NaN,NaN,NaN,"Washington, District of Columbia",Washington,District of Columbia,38.9047,-77.0163,"Washington, District of Columbia"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,900,434,"Speech- &quot;The 1965 Voting Rights Act, A Be...","Bond, Julian","Disenfranchisement, Race and Law",NaN,"Speech- &quot;The 1965 Voting Rights Act, A Be...",,NaN,NaN,NaN,NaN,NaN
383,902,503,"Commencement Address at Washington University,...","Bond, Julian",Education,NaN,"Commencement Address at Washington University,...","St. Louis, Missouri",St. Louis,Missouri,38.6359,-90.2451,"St. Louis, Missouri"
384,903,518,"Speech- &quot;Public Intellectuals &quot;, 12 ...","Bond, Julian","Education, Organizing",NaN,NaN,,NaN,NaN,NaN,NaN,NaN
385,904,566,Speech concerning politics and the debate surr...,"Bond, Julian","Economics, Organizing",NaN,(Draft) Speech concerning politics and the deb...,,NaN,NaN,NaN,NaN,NaN


In [173]:
merged.to_csv('speeches_to_ArcGIS.csv')

In [211]:
mystr = merged[merged['nid']==610]

In [129]:
doc_catalog['ID'] = doc_catalog['ID'].astype('str')
doc_catalog['url'] = "http://bondpapersproject.org/node/" + doc_catalog['ID']
doc_catalog

,ID,Title,Document Body,City,url
0,670,Undated Speech concerning Conditions of Black ...,[This speech includes pages with many differen...,NaN,http://bondpapersproject.org/node/670
1,667,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o...",NaN,http://bondpapersproject.org/node/667
2,666,Speeches making observations about the recent ...,"Now that the nation's voters — at least, 54% o...",NaN,http://bondpapersproject.org/node/666
3,665,Speeches making observations about the recent ...,"1\nNow that the nations voters — at least, 54%...",NaN,http://bondpapersproject.org/node/665
4,663,Speech about the upcoming presidential electio...,The election approaching on November seventh i...,"New Orleans, Louisiana",http://bondpapersproject.org/node/663
...,...,...,...,...,...
235,83,Speech concerning black bankers and businesses...,"Before I begin, let me tell you my financial q...",NaN,http://bondpapersproject.org/node/83
236,84,Speech concerning challenges facing the Americ...,Now that the nation has changed had a change o...,NaN,http://bondpapersproject.org/node/84
237,85,Speech from the Alabama Voter Education Projec...,Thank you a great deal. As John Lewis has told...,NaN,http://bondpapersproject.org/node/85
238,86,Speech at Press Conference in Washington conce...,[Handwritten.]\none the point of today's conce...,NaN,http://bondpapersproject.org/node/86


In [131]:
doc_catalog['url'][1]

'http://bondpapersproject.org/node/667'

So I think the conclusion here is that location tagging ought to be done during proofreading. I wish there was an easy way to make that location tagging happen, especially vis a vis creating the locations, but maybe this can be a jumping off point.

Essentially, there is no foolproof way to do this, but if the location is in the title, we can create entries for those and then build out from there. And hopefully, we can then go in and scrape that location data and apply it to the map tile layer.

city_from_body seems fairly useless

In [135]:
grouped = doc_catalog.groupby('City').agg({'Title': lambda x: ' | '.join(x),
                                           'ID': 'count',
                                           'url': lambda x: ' | '.join(x)
                                           }).reset_index()
grouped

,City,Title,ID,url
0,"Arlington, Texas","Speech--""Strategies for Change"" The Texas Regi...",1,http://bondpapersproject.org/node/336
1,"Atlanta, Georgia",Speech about the roles of black students in ad...,25,http://bondpapersproject.org/node/618 | http:/...
2,"Atlantic City, New Jersey",Speech given at the Philadelphia Bar Associati...,1,http://bondpapersproject.org/node/470
3,"Baltimore, Maryland",Commencement Address at Morgan State Universit...,1,http://bondpapersproject.org/node/442
4,"Birmingham, Alabama",Speech concerning opposition to the War in Sou...,2,http://bondpapersproject.org/node/213 | http:/...
5,"Charlotte, North Carolina",Speech prepared for Black Democrats Conference...,2,http://bondpapersproject.org/node/311 | http:/...
6,"Chicago, Illinois",Speech concerning the 1972 Presidential Electi...,1,http://bondpapersproject.org/node/118
7,"Cleveland, Ohio",Remarks prepared for Morehouse College Alumni ...,3,http://bondpapersproject.org/node/157 | http:/...
8,"Greensboro, North Carolina",Remarks to the Triad Sickle Cell Anemia Founda...,1,http://bondpapersproject.org/node/237
9,"Houston, Texas","Speech delivered at the NAACP Convention, Geor...",4,http://bondpapersproject.org/node/521 | http:/...


In [143]:
merged = pd.merge(grouped, uscities, on='City', how='left')
merged

,City,Title,ID,url,city,state_name,lat,lng
0,"Arlington, Texas","Speech--""Strategies for Change"" The Texas Regi...",1,http://bondpapersproject.org/node/336,Arlington,Texas,32.6998,-97.1250
1,"Atlanta, Georgia",Speech about the roles of black students in ad...,25,http://bondpapersproject.org/node/618 | http:/...,Atlanta,Georgia,33.7628,-84.4220
2,"Atlantic City, New Jersey",Speech given at the Philadelphia Bar Associati...,1,http://bondpapersproject.org/node/470,Atlantic City,New Jersey,39.3797,-74.4527
3,"Baltimore, Maryland",Commencement Address at Morgan State Universit...,1,http://bondpapersproject.org/node/442,Baltimore,Maryland,39.3051,-76.6144
4,"Birmingham, Alabama",Speech concerning opposition to the War in Sou...,2,http://bondpapersproject.org/node/213 | http:/...,Birmingham,Alabama,33.5279,-86.7971
5,"Charlotte, North Carolina",Speech prepared for Black Democrats Conference...,2,http://bondpapersproject.org/node/311 | http:/...,Charlotte,North Carolina,35.2083,-80.8303
6,"Chicago, Illinois",Speech concerning the 1972 Presidential Electi...,1,http://bondpapersproject.org/node/118,Chicago,Illinois,41.8375,-87.6866
7,"Cleveland, Ohio",Remarks prepared for Morehouse College Alumni ...,3,http://bondpapersproject.org/node/157 | http:/...,Cleveland,Ohio,41.4764,-81.6805
8,"Greensboro, North Carolina",Remarks to the Triad Sickle Cell Anemia Founda...,1,http://bondpapersproject.org/node/237,Greensboro,North Carolina,36.0956,-79.8271
9,"Houston, Texas","Speech delivered at the NAACP Convention, Geor...",4,http://bondpapersproject.org/node/521 | http:/...,Houston,Texas,29.7860,-95.3885


In [145]:
drop_map = ['city', 'state_name']
merged.drop(drop_map, axis=1, inplace=True)
merged.rename({'ID':'count', 'Title':'Documents'}, axis=1, inplace=True)
merged

,City,Documents,count,url,lat,lng
0,"Arlington, Texas","Speech--""Strategies for Change"" The Texas Regi...",1,http://bondpapersproject.org/node/336,32.6998,-97.1250
1,"Atlanta, Georgia",Speech about the roles of black students in ad...,25,http://bondpapersproject.org/node/618 | http:/...,33.7628,-84.4220
2,"Atlantic City, New Jersey",Speech given at the Philadelphia Bar Associati...,1,http://bondpapersproject.org/node/470,39.3797,-74.4527
3,"Baltimore, Maryland",Commencement Address at Morgan State Universit...,1,http://bondpapersproject.org/node/442,39.3051,-76.6144
4,"Birmingham, Alabama",Speech concerning opposition to the War in Sou...,2,http://bondpapersproject.org/node/213 | http:/...,33.5279,-86.7971
5,"Charlotte, North Carolina",Speech prepared for Black Democrats Conference...,2,http://bondpapersproject.org/node/311 | http:/...,35.2083,-80.8303
6,"Chicago, Illinois",Speech concerning the 1972 Presidential Electi...,1,http://bondpapersproject.org/node/118,41.8375,-87.6866
7,"Cleveland, Ohio",Remarks prepared for Morehouse College Alumni ...,3,http://bondpapersproject.org/node/157 | http:/...,41.4764,-81.6805
8,"Greensboro, North Carolina",Remarks to the Triad Sickle Cell Anemia Founda...,1,http://bondpapersproject.org/node/237,36.0956,-79.8271
9,"Houston, Texas","Speech delivered at the NAACP Convention, Geor...",4,http://bondpapersproject.org/node/521 | http:/...,29.7860,-95.3885


Of course, the next question is how to hyperlink them all, but perhaps that is one for later. Not bad work today.

I have some thoughts on how to hyperlink, too. Definitely not nothing.

Ok, so, cannot create a content field with a list of hyperlinks and have it work. Individual hyperlinks do work though - could create links to each place name and have links to content

In [147]:
merged.to_csv('test_layer_2.csv')

alternatively:

In [181]:
merged_pt_2 = pd.merge(doc_catalog, uscities, on='City', how='left')
merged_pt_2.dropna(inplace=True)
merged_pt_2

,ID,Title,Document Body,City,url,city,state_name,lat,lng
4,663,Speech about the upcoming presidential electio...,The election approaching on November seventh i...,"New Orleans, Louisiana",http://bondpapersproject.org/node/663,New Orleans,Louisiana,30.0687,-89.9288
5,662,(Draft) Speech about the upcoming presidential...,Page 1\nJULIAN BOND\nThe election approaching ...,"New Orleans, Louisiana",http://bondpapersproject.org/node/662,New Orleans,Louisiana,30.0687,-89.9288
6,661,(Draft) Speech about the upcoming presidential...,[Handwritten on legal pad paper.]\n[The follow...,"New Orleans, Louisiana",http://bondpapersproject.org/node/661,New Orleans,Louisiana,30.0687,-89.9288
11,618,Speech about the roles of black students in ad...,1\n[Handwritten.]\nI have been asked to say a ...,"Atlanta, Georgia",http://bondpapersproject.org/node/618,Atlanta,Georgia,33.7628,-84.4220
16,613,"Speech concerning political action, black peop...",It is a great pleasure to be in Florida this e...,"Sanford, Florida",http://bondpapersproject.org/node/613,Sanford,Florida,28.7891,-81.2757
...,...,...,...,...,...,...,...,...,...
221,90,"Speech at the National Urban League Meeting, K...",The quadrennial occasion for American reaction...,"Kansas City, Missouri",http://bondpapersproject.org/node/90,Kansas City,Missouri,39.1238,-94.5541
226,95,Testimony before the Commission on Delegate Se...,I am pleased to testify on behalf of the refor...,"Atlanta, Georgia",http://bondpapersproject.org/node/95,Atlanta,Georgia,33.7628,-84.4220
227,96,Testimony before the Commission on Delegate Se...,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia",http://bondpapersproject.org/node/96,Atlanta,Georgia,33.7628,-84.4220
228,97,Testimony before the Commission on Delegate Se...,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia",http://bondpapersproject.org/node/97,Atlanta,Georgia,33.7628,-84.4220


In [185]:
drop_map_2 = drop_map + ['Document Body']
merged_pt_2.drop(drop_map_2, axis=1, inplace=True)
merged_pt_2

,ID,Title,City,url,lat,lng
4,663,Speech about the upcoming presidential electio...,"New Orleans, Louisiana",http://bondpapersproject.org/node/663,30.0687,-89.9288
5,662,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",http://bondpapersproject.org/node/662,30.0687,-89.9288
6,661,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",http://bondpapersproject.org/node/661,30.0687,-89.9288
11,618,Speech about the roles of black students in ad...,"Atlanta, Georgia",http://bondpapersproject.org/node/618,33.7628,-84.4220
16,613,"Speech concerning political action, black peop...","Sanford, Florida",http://bondpapersproject.org/node/613,28.7891,-81.2757
...,...,...,...,...,...,...
221,90,"Speech at the National Urban League Meeting, K...","Kansas City, Missouri",http://bondpapersproject.org/node/90,39.1238,-94.5541
226,95,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",http://bondpapersproject.org/node/95,33.7628,-84.4220
227,96,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",http://bondpapersproject.org/node/96,33.7628,-84.4220
228,97,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",http://bondpapersproject.org/node/97,33.7628,-84.4220


In [187]:
merged_pt_2.to_csv('test_layer_3.csv')

# Using all the speeches

In [189]:
catalog_extended = pd.read_csv('document-catalog_extended.csv')
catalog_extended

,ID,Title,Box,Document Length,Workflow Stage,Image Filename,Image Identifier,Image URL,Document Body
0,4375,Speech concerning Black Voters and Officials d...,Box 15 Folder 1,6.0,Cataloged,NaN,NaN,NaN,NaN
1,3261,Enclosure: Julian Bond's revisions to text abo...,Box 14 Folder 6,1.0,Cataloged,NaN,NaN,NaN,NaN
2,671,"Speech File Material: Speech Fragment, 1971?",Box 12 Folder 3,4.0,Transcribed,"PJB671_mss13347-b12-f3_0088.tif\n, PJB671_mss1...","3243642, 3243643, 3243644, 3243645",NaN,"(9)\nNearly 100 years ago, a group of black pe..."
3,670,Undated Speech concerning Conditions of Black ...,Box 12 Folder 3,17.0,Early Access,"PJB670_mss13347-b12-f3_0071.tif\n, PJB670_mss1...","3243646, 3243647, 3243648, 3243649, 3243650, 3...",NaN,[This speech includes pages with many differen...
4,668,Speech File Material: Undated Speech about the...,Box 12 Folder 3,10.0,Cataloged,"PJB668_mss13347-b12-f3_0058.tif\n, PJB668_mss1...","3243666, 3243667, 3243668, 3243669, 3243670, 3...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
433,88,Speech on the health of the United States educ...,Box 4 Folder 8,29.0,Early Access,"PJB88_0001.tif\n, PJB88_0002.tif\n, PJB88_0003...","2857526, 2857527, 2857528, 2857529, 2857530, 2...",NaN,Insert x [Handwritten pages contain the conten...
434,4,"""Collision Course in a Divided America""",NaN,NaN,Digitized,NaN,NaN,NaN,"Unnamed speaker:\nIn the past few years, espec..."
435,7,"Address at the ""New Directions in Civil Rights...",NaN,NaN,Transcribed,NaN,NaN,NaN,This is the twentieth anniversary year of the ...
436,11,"DUPLICATE DO NOT PUBLISH: ""Civil Rights and Wr...",NaN,NaN,Digitized,"1578302.tiff\n, 1578303.tiff","1578302, 1578303",NaN,The women’s movement was given new life in the...


In [191]:
catalog_extended.drop(['Box', 'Document Length',
                  'Image Filename', 'Image Identifier', 'Image URL'], axis=1, inplace=True)

In [193]:
catalog_extended['City'] = catalog_extended['Title'].str.extract(f'({city_pattern})', expand=False)

sum(catalog_extended.City.value_counts())

97

EDA unfortunately shows that it will be awfully difficult to extract the dates because they are not in the same pattern.

In [227]:
years = np.arange(1960, 2016)
years = years.astype('str')
years_list = list(years)
years_pattern = '|'.join([f'\\b{year}\\b' for year in years])
years_pattern

'\\b1960\\b|\\b1961\\b|\\b1962\\b|\\b1963\\b|\\b1964\\b|\\b1965\\b|\\b1966\\b|\\b1967\\b|\\b1968\\b|\\b1969\\b|\\b1970\\b|\\b1971\\b|\\b1972\\b|\\b1973\\b|\\b1974\\b|\\b1975\\b|\\b1976\\b|\\b1977\\b|\\b1978\\b|\\b1979\\b|\\b1980\\b|\\b1981\\b|\\b1982\\b|\\b1983\\b|\\b1984\\b|\\b1985\\b|\\b1986\\b|\\b1987\\b|\\b1988\\b|\\b1989\\b|\\b1990\\b|\\b1991\\b|\\b1992\\b|\\b1993\\b|\\b1994\\b|\\b1995\\b|\\b1996\\b|\\b1997\\b|\\b1998\\b|\\b1999\\b|\\b2000\\b|\\b2001\\b|\\b2002\\b|\\b2003\\b|\\b2004\\b|\\b2005\\b|\\b2006\\b|\\b2007\\b|\\b2008\\b|\\b2009\\b|\\b2010\\b|\\b2011\\b|\\b2012\\b|\\b2013\\b|\\b2014\\b|\\b2015\\b'

In [291]:
catalog_extended['Year'] = catalog_extended['Title'].str.extract(f'({years_pattern})', expand=False)
catalog_extended['Year'].value_counts()

Year
1977    17
1972     9
1975     8
1976     8
1971     6
1979     5
1970     4
1980     4
1981     3
1978     3
1973     3
1998     3
1991     2
1974     2
1983     2
2001     2
1989     2
1965     1
2003     1
2002     1
2000     1
1996     1
1999     1
1994     1
1988     1
1968     1
1995     1
1987     1
Name: count, dtype: int64

In [293]:
catalog_extended.dropna(inplace=True)
catalog_extended.Year.value_counts()

Year
1977    17
1972     9
1975     8
1976     8
1971     6
1979     5
1970     4
1980     4
1981     3
1978     3
1973     3
1998     3
1991     2
1974     2
1983     2
2001     2
1989     2
1965     1
2003     1
2002     1
2000     1
1996     1
1999     1
1994     1
1988     1
1968     1
1995     1
1987     1
Name: count, dtype: int64

In [295]:
merged_cat = pd.merge(catalog_extended, uscities, on='City', how='left')
merged_cat.drop(['Document Body', 'city', 'state_name'], axis=1, inplace=True)
merged_cat['ID'] = merged_cat['ID'].astype('str')
merged_cat['url'] = "http://bondpapersproject.org/node/" + merged_cat['ID']
merged_cat

,ID,Title,City,Year,lat,lng,url
0,663,Speech about the upcoming presidential electio...,"New Orleans, Louisiana",1972,30.0687,-89.9288,http://bondpapersproject.org/node/663
1,662,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",1972,30.0687,-89.9288,http://bondpapersproject.org/node/662
2,661,(Draft) Speech about the upcoming presidential...,"New Orleans, Louisiana",1972,30.0687,-89.9288,http://bondpapersproject.org/node/661
3,618,Speech about the roles of black students in ad...,"Atlanta, Georgia",1972,33.7628,-84.4220,http://bondpapersproject.org/node/618
4,613,"Speech concerning political action, black peop...","Sanford, Florida",1972,28.7891,-81.2757,http://bondpapersproject.org/node/613
...,...,...,...,...,...,...,...
89,89,"Speech at the National Urban League Meeting, K...","Kansas City, Missouri",1974,39.1238,-94.5541,http://bondpapersproject.org/node/89
90,90,"Speech at the National Urban League Meeting, K...","Kansas City, Missouri",1974,39.1238,-94.5541,http://bondpapersproject.org/node/90
91,95,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",1973,33.7628,-84.4220,http://bondpapersproject.org/node/95
92,96,Testimony before the Commission on Delegate Se...,"Atlanta, Georgia",1973,33.7628,-84.4220,http://bondpapersproject.org/node/96


In [309]:
merged_cat['Year'] = merged_cat['Year'].astype('str')

In [1]:
mydict = {}
for year in merged_cat['Year'].unique():
    mydict[year] = merged_cat[merged_cat['Year'] == year]

filtered_dict = {k: v for k, v in mydict.items() if k in np.arange(1970,1980).astype('str')}
filtered_dict = dict(sorted(filtered_dict.items()))

filtered_dict.head()

NameError: name 'merged_cat' is not defined

In [313]:
for k, v in filtered_dict.items():
    path = f'speeches_{str(k)}.csv'
    v.to_csv(path)

In [317]:
frames = filtered_dict.values()
speeches_70s = pd.concat(frames)
speeches_70s.to_csv('speeches_70s.csv')

Next week's puzzle (or maybe this weekend): try to get state abbreviations (CA, AL, etc.)

1. Create a second state string to extract based on. So, I would do an str.replace with a map to each state's abbreviation on the uscities dataset: would mean I had like "Huntsville, Alabama" but also "Huntsville, AL".
2. Then I can add both of those to the regex function and get it all in one column (some will be "Huntsville, AL" and others will be "Huntsville, Alabama")
4. From there, I can use the same replace map I made in the first place to loop through that column and convert all like "Huntsville, Alabama" to "Huntsville, AL".
5. Now, I can join based on that column in the uscities dataset, and go from there. Once I drop the columns, the entries will look exactly the same as before, just hopefully with even more information.
6. I think I am going to create a new Jupyter notebook for this and work on it on Sunday while I watch football. The new Jupyter notebook will allow me to clean up the mess I have made so far.

In [321]:
uscities

,city,state_name,lat,lng,City
0,New York,New York,40.6943,-73.9249,"New York, New York"
1,Los Angeles,California,34.1141,-118.4068,"Los Angeles, California"
2,Chicago,Illinois,41.8375,-87.6866,"Chicago, Illinois"
3,Miami,Florida,25.7840,-80.2101,"Miami, Florida"
4,Houston,Texas,29.7860,-95.3885,"Houston, Texas"
...,...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032,"Kohatk, Arizona"
31116,Ironville,Pennsylvania,40.6586,-78.2155,"Ironville, Pennsylvania"
31117,Newkirk,New Mexico,35.0635,-104.2715,"Newkirk, New Mexico"
31118,Falcon Village,Texas,26.5652,-99.1341,"Falcon Village, Texas"


In [329]:
state_map = pd.read_csv('states.csv')
state_map

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [339]:
state_map_dict = dict(zip(state_map['State'], state_map['Abbreviation']))
uscities['state_abbrev'] = uscities['state_name'].map(state_map_dict)
uscities

,city,state_name,lat,lng,City,state_abbrev
0,New York,New York,40.6943,-73.9249,"New York, New York",NY
1,Los Angeles,California,34.1141,-118.4068,"Los Angeles, California",CA
2,Chicago,Illinois,41.8375,-87.6866,"Chicago, Illinois",IL
3,Miami,Florida,25.7840,-80.2101,"Miami, Florida",FL
4,Houston,Texas,29.7860,-95.3885,"Houston, Texas",TX
...,...,...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032,"Kohatk, Arizona",AZ
31116,Ironville,Pennsylvania,40.6586,-78.2155,"Ironville, Pennsylvania",PA
31117,Newkirk,New Mexico,35.0635,-104.2715,"Newkirk, New Mexico",NM
31118,Falcon Village,Texas,26.5652,-99.1341,"Falcon Village, Texas",TX


In [359]:
uscities.rename({'City': 'city_full'}, axis = 1, inplace = True)

uscities['city_abbrev'] = uscities['city'] + ', ' + uscities['state_abbrev']
uscities

,city,state_name,lat,lng,city_full,state_abbrev,city_abbrev
0,New York,New York,40.6943,-73.9249,"New York, New York",NY,"New York, NY"
1,Los Angeles,California,34.1141,-118.4068,"Los Angeles, California",CA,"Los Angeles, CA"
2,Chicago,Illinois,41.8375,-87.6866,"Chicago, Illinois",IL,"Chicago, IL"
3,Miami,Florida,25.7840,-80.2101,"Miami, Florida",FL,"Miami, FL"
4,Houston,Texas,29.7860,-95.3885,"Houston, Texas",TX,"Houston, TX"
...,...,...,...,...,...,...,...
31115,Kohatk,Arizona,32.5781,-112.0032,"Kohatk, Arizona",AZ,"Kohatk, AZ"
31116,Ironville,Pennsylvania,40.6586,-78.2155,"Ironville, Pennsylvania",PA,"Ironville, PA"
31117,Newkirk,New Mexico,35.0635,-104.2715,"Newkirk, New Mexico",NM,"Newkirk, NM"
31118,Falcon Village,Texas,26.5652,-99.1341,"Falcon Village, Texas",TX,"Falcon Village, TX"


In [369]:

cities = uscities['city_full'].tolist() + uscities['city_abbrev'].tolist()

# Create regex pattern for cities
city_pattern = '|'.join([f'\\b{city}\\b' for city in cities])

In [377]:
# Extract the city and state from the Address column
catalog_extended = pd.read_csv('document-catalog_extended.csv')

catalog_extended['city_full'] = catalog_extended['Title'].str.extract(f'({city_pattern})', expand=False)

print(catalog_extended)

       ID                                              Title              Box  \
0    4375  Speech concerning Black Voters and Officials d...  Box 15 Folder 1   
1    3261  Enclosure: Julian Bond's revisions to text abo...  Box 14 Folder 6   
2     671       Speech File Material: Speech Fragment, 1971?  Box 12 Folder 3   
3     670  Undated Speech concerning Conditions of Black ...  Box 12 Folder 3   
4     668  Speech File Material: Undated Speech about the...  Box 12 Folder 3   
..    ...                                                ...              ...   
433    88  Speech on the health of the United States educ...   Box 4 Folder 8   
434     4            "Collision Course in a Divided America"              NaN   
435     7  Address at the "New Directions in Civil Rights...              NaN   
436    11  DUPLICATE DO NOT PUBLISH: "Civil Rights and Wr...              NaN   
437    12  Speech concerning the black college: its effec...              NaN   

     Document Length Workfl

In [379]:
catalog_extended.drop(['Box', 'Document Length',
                  'Image Filename', 'Image Identifier', 'Image URL'], axis=1, inplace=True)

In [387]:
filtered_df = catalog_extended[catalog_extended['city_full'].notnull()]
filtered_df

,ID,Title,Workflow Stage,Document Body,city_full
4,668,Speech File Material: Undated Speech about the...,Cataloged,NaN,"Chicago, Illinois"
8,663,Speech about the upcoming presidential electio...,Early Access,The election approaching on November seventh i...,"New Orleans, Louisiana"
9,662,(Draft) Speech about the upcoming presidential...,Early Access,Page 1\nJULIAN BOND\nThe election approaching ...,"New Orleans, Louisiana"
10,661,(Draft) Speech about the upcoming presidential...,Early Access,[Handwritten on legal pad paper.]\n[The follow...,"New Orleans, Louisiana"
11,660,Speech concerning the trial of the RNA–11 (Rep...,Early Access,"Writing of another political trial, his own, D...","Jackson, MS"
...,...,...,...,...,...
415,90,"Speech at the National Urban League Meeting, K...",Early Access,The quadrennial occasion for American reaction...,"Kansas City, Missouri"
420,95,Testimony before the Commission on Delegate Se...,Early Access,I am pleased to testify on behalf of the refor...,"Atlanta, Georgia"
421,96,Testimony before the Commission on Delegate Se...,Early Access,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia"
422,97,Testimony before the Commission on Delegate Se...,Early Access,"""Quotas"" is the code word for 1972. Like most ...","Atlanta, Georgia"


In [389]:
for abbrev, full_name in state_map_dict.items():
    filtered_df['city_full'] = filtered_df['city_full'].str.replace(full_name, abbrev)

print(filtered_df)

      ID                                              Title Workflow Stage  \
4    668  Speech File Material: Undated Speech about the...      Cataloged   
8    663  Speech about the upcoming presidential electio...   Early Access   
9    662  (Draft) Speech about the upcoming presidential...   Early Access   
10   661  (Draft) Speech about the upcoming presidential...   Early Access   
11   660  Speech concerning the trial of the RNA–11 (Rep...   Early Access   
..   ...                                                ...            ...   
415   90  Speech at the National Urban League Meeting, K...   Early Access   
420   95  Testimony before the Commission on Delegate Se...   Early Access   
421   96  Testimony before the Commission on Delegate Se...   Early Access   
422   97  Testimony before the Commission on Delegate Se...   Early Access   
426   80  Speech made before the Hungry Club [Forum, Atl...   Early Access   

                                         Document Body         

C:\Users\charl\AppData\Local\Temp\ipykernel_23116\1392622918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['city_full'] = filtered_df['city_full'].str.replace(full_name, abbrev)


hmmm. Setting With Copy warning is decidedly a concern, but not sure how big of a concern it ought to be. The dataset's so small, and I can always just filter the way I had filtered before... trying to remember why I want to keep it. Every doc has a workflow stage, right?

Oh also none of this will matter once the other interns start doing location tagging. Duh. Main deliverable should just be a way to tag places. Sigh.

In [399]:
filtered_df.loc[415,:]

ID                                                               90
Title             Speech at the National Urban League Meeting, K...
Workflow Stage                                         Early Access
Document Body     The quadrennial occasion for American reaction...
city_full                                     Kansas City, Missouri
Name: 415, dtype: object